# __TP1 - Classificador Supervisionado__

O objetivo central deste trabalho prático é desenvolver um classificador supervisionado utilizando conceitos de geometria computacional para cumprir essa tarefa.

---

#### __Disciplina:__ Algoritmos II

#### __Grupo:__
Vinicius Silva Gomes - 2021421869

Mirna

Daniel

In [2]:
# Importando bibliotecas e configurações iniciais

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functools import cmp_to_key

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import sys

plt.rcParams['figure.figsize']  = (16, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4

plt.ion()
plt.style.use('seaborn-colorblind')

In [3]:
# URL dos Datasets

WINES = "../data/csv/wine.csv"
TIC_TAC_TOE = "../data/csv/tic-tac-toe.csv"
MUSHROOM = "../data/csv/mushroom.csv"
MAGIC = "../data/csv/magic.csv"
IRIS = "../data/csv/iris.csv"
CHESS = "../data/csv/chess.csv"
CAR = "../data/csv/car.csv"
BANANA = "../data/csv/banana.csv"
LETTER = "../data/csv/letter.csv"
TITANIC = "../data/csv/titanic.csv"

In [4]:
# Manipulação dos datasets

wines_df = pd.read_csv(WINES, sep=',')
tic_tac_toe_df = pd.read_csv(TIC_TAC_TOE, sep=',')
mushroom_df = pd.read_csv(MUSHROOM, sep=',')
magic_df = pd.read_csv(MAGIC, sep=',')
iris_df = pd.read_csv(IRIS, sep=',')
chess_df = pd.read_csv(CHESS, sep=',')
car_df = pd.read_csv(CAR, sep=',')
banana_df = pd.read_csv(BANANA, sep=',')
letter_df = pd.read_csv(LETTER, sep=',')
titanic_df = pd.read_csv(TITANIC, sep=',')

In [34]:
# Definição de classes

class Point:
    def __init__(self, x = None, y = None):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f"({self.x}, {self.y})"

In [55]:
# Métodos auxiliares

def relative_position(p, q, r):
    # Produto vetorial dos segmentos pq e pr
    pos = ((q.y - p.y) * (r.x - q.x)) - ((q.x - p.x) * (r.y - q.y))
    
    if pos == 0:
        return 0 # colinear
    elif pos > 0:
        return 1 # sentido horário
    else:
        return -1 # sentido antihorário

## Envoltória Convexa

def find_minor_point(points):
    index = -1
    min_y = sys.maxsize
    
    for idx, point in enumerate(points):
        if point.y < min_y:
            index = idx
            min_y = point.y
        elif point.y == min_y:
            if point.x < points[index].x:
                index = idx
                
    return (points[index], index)

def euclidean_distance(p1, p2):
    distance = (p1.x - p2.x) ** 2 + ((p1.y - p2.y) ** 2)
    
    return distance
    
def polar_sort(p0, p1, p2):
    relative_pos = relative_position(p0, p1, p2)
    
    if relative_pos == 0:
        if euclidean_distance(p0, p1) < euclidean_distance(p0, p2):
            return -1
        else:
            return 1
    else:
        if relative_pos < 0:
            return -1
        else:
            return 1
    
def next_to_top(hull):
    return hull[-2]

def top(hull):
    return hull[-1]

## Varredura linear

# Verifica se um ponto está contido dentro de um segmento
def on_segment(p, q, r):
    # Produto vetorial dos segmentos pq e pr
    pos = ((q.x - p.x) * (r.y - p.y)) - ((q.y - p.y) * (r.x - p.x))
    
    if pos == 0:
        return True
    else:
        return False

# Verifica se dois segmentos se intersectam
def segments_intersect(p1, p2, p3, p4):
    d1 = relative_position(p3, p4, p1)
    d2 = relative_position(p3, p4, p2)
    d3 = relative_position(p1, p2, p3)
    d4 = relative_position(p1, p2, p4)
    
    if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and ((d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0)):
        return True
    elif d1 == 0 and on_segment(p3, p4, p1):
        return True
    elif d2 == 0 and on_segment(p3, p4, p2):
        return True
    elif d3 == 0 and on_segment(p1, p2, p3):
        return True
    elif d4 == 0 and on_segment(p1, p2, p4):
        return True
    else:
        return False

In [56]:
# Métodos principais

# Calcula a envoltória convexa para o conjunto de pontos dados
def graham_scan(points):
    # Encontra p0 no conjunto de dados
    p0, idx = find_minor_point(points)
    
    # Troca o p0 de lugar elemento de índice 0 do conjunto
    points[0], points[idx] = points[idx], points[0]
    
    # Ordena os elementos restantes a partir da coordenada polar com relação ao elemento p0
    sorted_points = sorted(points[1:], key=cmp_to_key(lambda p1, p2: polar_sort(p0, p1, p2)))
    
    # Remove os pontos que possuem o mesmo ângulo polar em relação a p0 (mantém o que está mais longe)
    to_remove = []
    
    for i in range(len(sorted_points) - 1):
        pos = relative_position(sorted_points[i], sorted_points[i + 1], p0)
        
        if pos == 0:
            to_remove.append(i)
    
    sorted_points = [i for j, i in enumerate(sorted_points) if j not in to_remove]
    
    if(len(sorted_points) < 2):
        print("Nao vai ta tendo como")
        return
    
    hull = []
    hull.append(p0)
    hull.append(sorted_points[0])
    hull.append(sorted_points[1])
    
    for i in range(2, len(sorted_points)):
        while ((len(hull) > 1) and (relative_position(next_to_top(hull), top(hull), points[i]) != -1)):
            hull.pop()
            
        hull.append(points[i])
    
    return hull

# Algoritmo da varredura linear para detectar interseção entre um conjunto de pontos
# def line_sweep(points):
    
# Algoritmo para verificar se há interseção entre as envoltórias convexas
# def check_intersection():

In [57]:
# Classificação

points = [Point(1, 4), Point(3, 3), Point(3, 1), Point(0, 0), Point(9, 6), Point(5, 5), Point(7, 0), Point(5, 2)]

hull = graham_scan(points)

for i in hull:
    print(i)

(0, 0)
(7, 0)
(9, 6)
(5, 5)
